In [1]:
import numpy as np

In [2]:
def compare(fitness_a, fitness_b):
    # if a dominates b, return will be 1, -1 if the contrary
    # if there is no dominance, 0 will be returned
    try:
        m = (fitness_a[0] - fitness_b[0]) / (fitness_a[1] - fitness_b[1])
    except ZeroDivisionError:
        return 0
    if m > 0:
        if fitness_a[0] < fitness_b[0]:
            return 1
        return -1
    return 0


In [3]:
def get_parents(parent_F:list):
	levels  = np.array(parent_F)[:, 2]
	max_idx = np.argmin(levels)
	return parent_F[max_idx]

In [4]:
def mix_parents(parent_a:list, parent_b:list, ratio:float = 0.5, get_indexes:bool=False):
	genotype_len = len(parent_a)

	a_idxs = set()
	acum = 0
	for i in range(genotype_len):
		acum += ratio
		if acum >= 1:
			acum -= 1
			a_idxs.add(i)
	b_idxs = {i for i in range(genotype_len)} - a_idxs

	a_idxs = list(a_idxs)
	b_idxs = list(b_idxs)
	child  = np.zeros((genotype_len), bool)

	child[a_idxs] = parent_a[a_idxs].copy()
	child[b_idxs] = parent_b[b_idxs].copy()
	if get_indexes:
		return child, (a_idxs, b_idxs)
	return child